In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
import gensim
from gensim.models import Word2Vec
import nltk 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
stop=set(stopwords.words('english'))
import re
df=pd.read_csv("Q2TrueFakeNewsDataset.csv")
def transform(text):
    t=re.sub(r"[^a-zA-Z\s]","",text)
    token=word_tokenize(t.lower())
    token=[stemmer.stem(i) for i in token if i not in stop]
    return " ".join(token)

    
df['clean']=df['TEXT'].apply(transform)

In [5]:
df

,TEXT,Class,clean
0,WASHINGTON (Reuters) - The head of a conservat...,TRUE,washington reuter head conserv republican fact...
1,WASHINGTON (Reuters) - Transgender people will...,TRUE,washington reuter transgend peopl allow first ...
2,WASHINGTON (Reuters) - The special counsel inv...,TRUE,washington reuter special counsel investig lin...
3,WASHINGTON (Reuters) - Trump campaign adviser ...,TRUE,washington reuter trump campaign advis georg p...
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,TRUE,seattlewashington reuter presid donald trump c...
...,...,...,...
9994,It s no secret Republicans are salivating to f...,FAKE,secret republican saliv find someth anyth take...
9995,Republicans are about to lose a huge source of...,FAKE,republican lose huge sourc fund justic depart ...
9996,A pawn working for Donald Trump claimed that w...,FAKE,pawn work donald trump claim women respect vot...
9997,Fox News is desperate to sabotage Hillary Clin...,FAKE,fox news desper sabotag hillari clinton campai...


In [6]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df['Class']=label.fit_transform(df['Class'])


In [7]:
X_train,X_test,y_train,y_test=train_test_split(df['clean'],df['Class'],test_size=0.2,random_state=42)

In [9]:
data=[sent.split() for sent in X_train]
w2model=Word2Vec(data,vector_size=100,window=5)
wordvector=w2model.wv

In [10]:
def vectorizer(text):
    words=text.split()
    vectors = [wordvector[word] for word in words if word in wordvector]
    return np.mean(vectors,axis=0) if vectors else np.zeros(100)

x_trainw2v=np.array([vectorizer(text) for text in X_train])
x_testw2v=np.array([vectorizer(text) for text in X_test])
x_trainw2v.shape,y_train.shape

((7999, 100), (7999,))

In [54]:
model=Sequential([
    Dense(128,activation='relu',input_shape=(100,)),
    Dense(64,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,249 (83.00 KB)

 Trainable params: 21,249 (83.00 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])
model.fit(x_trainw2v,y_train,epochs=5,batch_size=4)

Epoch 1/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9592 - loss: 0.1136
Epoch 2/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9846 - loss: 0.0437
Epoch 3/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9889 - loss: 0.0346
Epoch 4/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9902 - loss: 0.0278
Epoch 5/5
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9900 - loss: 0.0264


In [56]:
x_trainw2v

array([[-0.36015213,  0.22015083,  0.14862385, ..., -0.13474348,
        -0.09297694,  0.17223404],
       [-0.22597519, -0.1532465 , -0.18582734, ..., -0.92986083,
        -0.17110011, -0.11707507],
       [-0.21714753, -0.22170371, -0.26233295, ..., -0.7632935 ,
        -0.27916673, -0.34645626],
       ...,
       [-0.42463106,  0.36755258,  0.05741316, ..., -0.06040965,
         0.00622489,  0.01698539],
       [ 0.27752843,  0.09643646, -0.1026919 , ..., -0.36510885,
        -0.29410315,  0.37260035],
       [ 0.10262414,  0.36820433, -0.07211433, ..., -0.03501737,
        -0.2584216 ,  0.3936316 ]], dtype=float32)